In [3]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 38.4 MB/s eta 0:00:0000:0100:01


Shorten the video into 1 minute

In [9]:
# Load the video
input_video_path = "/kaggle/input/gesturevideo/video_original.mp4"
cap = cv2.VideoCapture(input_video_path)

# Get the video dimensions, FPS, and calculate the total number of frames for the first minute
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
frames_to_keep = 60 * fps  # Keep only the first minute

# Initialize the VideoWriter
output_video_path = "video_cut.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened() and frame_count < frames_to_keep:
    ret, frame = cap.read()

    if not ret:
        break

    # Write the frame to the output video
    out.write(frame)

    frame_count += 1

# Release resources
cap.release()
out.release()

Process video to get openness, total movement, and leaning direction

In [18]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import ConvexHull

# Initialize MediaPipe's Holistic module
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Function to calculate the Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

# Function to calculate the openness of a pose
def pose_openness(holistic_landmarks):
    keypoints = [
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER],
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER],
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP],
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP],
    ]
    
    coords = np.array([(kp.x, kp.y) for kp in keypoints])
    hull = ConvexHull(coords)
    
    return hull.volume

# Function to calculate leaning direction
def leaning_direction(holistic_landmarks):
    nose = holistic_landmarks.landmark[mp_holistic.PoseLandmark.NOSE]
    left_shoulder = holistic_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = holistic_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
    
    avg_shoulder_z = (left_shoulder.z + right_shoulder.z) / 2

    if nose.z < avg_shoulder_z:
        return "Forward"
    else:
        return "Backward"
    
# Load the video
video_path = "/kaggle/working/video_cut.mp4"
cap = cv2.VideoCapture(video_path)

# Get the video dimensions and FPS
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize the VideoWriter
output_filename = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # You can also use "XVID" or "MJPG" for AVI files
out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

# Initialize variables
prev_landmarks = None
total_movement = 0
movement_threshold = 0.001  # Adjust the threshold to fine-tune movement detection sensitivity
keypoints_to_track = [
    mp_holistic.PoseLandmark.LEFT_WRIST,
    mp_holistic.PoseLandmark.RIGHT_WRIST,
    mp_holistic.PoseLandmark.LEFT_ANKLE,
    mp_holistic.PoseLandmark.RIGHT_ANKLE,
]

# Process the video frames
count = 0 
with mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe's Holistic module
        results = holistic.process(frame_rgb)

        # Draw holistic landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            # Calculate the total movement
            if prev_landmarks:
                frame_movement = 0
                for kp in keypoints_to_track:
                    distance = euclidean_distance(results.pose_landmarks.landmark[kp], prev_landmarks.landmark[kp])
                    frame_movement += distance

                if frame_movement > movement_threshold:
                    total_movement += frame_movement

            prev_landmarks = results.pose_landmarks
            # Calculate and display the total movement and pose openness on the frame
            openness_value = pose_openness(results.pose_landmarks)
            cv2.putText(frame, f"Total Movement: {total_movement:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, f"Pose Openness: {openness_value:.4f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            # Calculate and display the leaning direction
            leaning_dir = leaning_direction(results.pose_landmarks)
            cv2.putText(frame, f"Leaning: {leaning_dir}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Save the frame
        cv2.imwrite('frame' + str(count) + '.jpg', frame)
        out.write(frame)
    out.release()